In [1]:
import numpy as np
import pickle
import sklearn.feature_extraction.text as text_a
import re
from collections import Counter
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.decomposition import PCA
from tqdm import tqdm
from scipy import spatial
from nltk.corpus import stopwords
import json
from nltk.cluster.kmeans import KMeansClusterer
from scipy.spatial.distance import cosine

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
texts = pickle.load(open("answer.pck", "rb"))

In [4]:
tuples = pickle.load(open("tuples (2).pck", "rb"))

In [5]:
for i in range(len(texts)):
    texts[i] = texts[i][texts[i].replace("."," ").index("Responsibilities"):][16:-152]

In [6]:
coursera = pickle.load(open("coursera.p", "rb"))

In [9]:
coursera_texts = [i["description"] for i in coursera]

In [10]:
def clear_text(text):
    global lemmatizer
    
    tmp_text = ""
    
    text = text.lower().replace("\n", " ")
    
    for char in "!@#$%^&*()[]{};:,./<>?|`~-=_+●\\":
        text = text.replace(char, " ")
        
    text = re.sub('[^A-Za-z]+ ', '', text).split(" ")
    
    for word in text:
        tmp_text += lemmatizer.lemmatize(word, 'v') + " "
    
    return tmp_text

In [11]:
nltk.data.path.append("/home")

In [12]:
texts = list(map(clear_text, texts))

In [13]:
coursera_texts = list(map(clear_text, coursera_texts))

In [14]:
tmp_text = ""

for text in texts:
    tmp_text += text + " "

job_words = set(tmp_text.split(" "))

In [15]:
tmp_text = ""

for text in coursera_texts:
    tmp_text += text + " "

coursera_words = set(tmp_text.split(" "))

In [16]:
white_list = job_words & coursera_words

In [17]:
tmp = []

for text in tqdm(texts):
    tmp_text = ""
    for word in text.split(" "):
        if word in white_list and word not in set(stopwords.words('english')):
            tmp_text += word + " "

    tmp.append(tmp_text)

texts = tmp

100%|██████████| 835/835 [00:13<00:00, 59.88it/s]


In [18]:
tmp = []

for text in tqdm(coursera_texts):
    tmp_text = ""
    for word in text.split(" "):
        if word in white_list and word not in set(stopwords.words('english')):
            tmp_text += word + " "

    tmp.append(tmp_text)

coursera_texts = tmp

100%|██████████| 1499/1499 [00:16<00:00, 93.80it/s]


In [19]:
all_texts = texts + coursera_texts

In [20]:
vectorizer = text_a.TfidfVectorizer()

In [21]:
dtm = vectorizer.fit_transform(all_texts).toarray()

In [22]:
cluster = KMeansClusterer(10, spatial.distance.cosine)
cluster.cluster(dtm)

/anaconda3/lib/python3.5/site-packages/scipy/spatial/distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [23]:
topic_words = []

In [24]:
doctopic = [cluster.classify(i) for i in dtm]

/anaconda3/lib/python3.5/site-packages/scipy/spatial/distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [25]:
Counter(doctopic)

Counter({0: 348,
         1: 630,
         2: 180,
         3: 489,
         4: 70,
         5: 266,
         6: 81,
         7: 39,
         8: 161,
         9: 70})

In [26]:
topic_names = [i[1] for i in topic_words]

In [27]:
topics = [np.argmax(i) for i in doctopic]

In [28]:
jobs_topic_names = pickle.load(open("job_topics.pck", "rb"))

In [29]:
coursera_topic_names = [i["title"] for i in coursera]

In [30]:
len(jobs_topic_names+coursera_topic_names)

2337

In [31]:
from collections import defaultdict

In [32]:
clusters = defaultdict(lambda: [])

In [33]:
for a,b in zip(topics,[i + " (job)" for i  in jobs_topic_names] +[i + " (course)" for i in coursera_topic_names]):
    clusters[a].append((b, " (course)" in b))

In [34]:
cluster_courses = []
cluster_jobs = []

In [35]:
for a,b in enumerate(clusters):
    tmp_courses = []
    tmp_jobs = []
    
    if sum([i[1] for i in clusters[b]]) > 0:
        for k in clusters[b]:
            if k[1] == True:
                tmp_courses.append(k[0])
            else:
                tmp_jobs.append(k[0])
        cluster_courses.append(tmp_courses)
        cluster_jobs.append(tmp_jobs)

In [36]:
api_dict = {}

In [37]:
for job_list,courses_list  in zip(cluster_jobs,cluster_courses):
    for job in job_list:
        api_dict[job] = courses_list

In [38]:
ALL_JOBS = json.dumps([tuples[j.replace(" (job)", "")] for j in list(api_dict.keys())])

In [39]:
from flask import Flask
app = Flask(__name__)

In [40]:
@app.route("/course/<job>")
def get_courses(job):
    return json.dumps(api_dict[job])

In [41]:
@app.route("/all_jobs")
def hello():
    return ALL_JOBS

In [42]:
coursera = pickle.load(open('coursera.p', 'rb'))

def get_meta(recommendations):
    meta = [x for x in coursera if x['title'] in recommendations]
    result = {}
    # все из специализации для каждого в начало
    for course in meta:
        if 'specialization' in course:
            for insert in list(filter(
                    lambda x: 'specialization' in x and x['specialization'] == course['specialization'],
                    coursera)):
                result[insert['title']] = insert
    for course in meta:
        if 'specialization' not in course:
            result[course['title']] = course
    return result

In [ ]:
@app.route('recommend/<vector>&<job>'):
def recommend(vector, job)
    return (vector, job)

In [ ]:
app.run(host="0.0.0.0", port=8080)